In [2]:
from pycocotools.coco import COCO
import os
import torch
import torchvision
root = 'E:/Resource/Dataset/COCO/SubCOCO'
annDir = os.path.join(root, 'annotations/instances_{}.json')
# coco = COCO(annDir.format('train2017'))

## Creating Datasets

In [5]:
coco_train = torchvision.datasets.CocoDetection(os.path.join(root, 'train2017'),
                                         annDir.format('train2017'))
coco_val = torchvision.datasets.CocoDetection(os.path.join(root, 'val2017'),
                                         annDir.format('val2017'))
print('Amount of train images:', len(coco_train))
print('Amount of validation images:', len(coco_val))
print(coco_train[0])

loading annotations into memory...
Done (t=2.14s)
creating index...
index created!
loading annotations into memory...
Done (t=0.30s)
creating index...
index created!
Amount of train images: 19759
Amount of validation images: 870
(<PIL.Image.Image image mode=RGB size=480x640 at 0x115751EAF48>, [{'segmentation': [[52.36, 387.77, 100.68, 391.48, 123.81, 395.2, 124.23, 452.61, 130.01, 456.74, 128.77, 502.58, 127.94, 522.11, 52.36, 512.62], [145.7, 402.63, 179.16, 426.59, 180.81, 484.82, 174.61, 489.37, 171.31, 504.92, 173.79, 507.81, 175.03, 540.85, 169.24, 544.15, 157.27, 545.39, 146.53, 540.43, 142.4, 537.96, 138.27, 534.24, 137.03, 532.17, 137.03, 515.65, 137.03, 457.83, 140.33, 454.94, 141.99, 402.9], [206.42, 432.64, 228.72, 435.94, 213.85, 490.05, 208.48, 490.46, 210.13, 483.44, 208.07, 483.44]], 'area': 15185.179800000002, 'iscrowd': 0, 'image_id': 64, 'bbox': [52.36, 387.77, 176.36, 157.62], 'category_id': 3, 'id': 350246}, {'segmentation': [[127.76, 396.98, 98.34, 389.46, 52.64, 3

## Defining Model

In [ ]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_instance_segmentation_model(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                      hidden_layer,
                                                      num_classes)
    return model